### 【 데이터응용 - 함수 매핑 】

- pandas의 apply(), map(), pipe(), transform() 
- 모두 데이터를 함수에 전달해 변환하는 메서드
- 적용 범위와 반환 형식이 모두 다름

In [2]:
## 모듈 로딩
import pandas as pd
import numpy as np

In [3]:
## 사용자 정의 함수
def test(n):
    print(f'test() - {type(n)}, {n}')
    if isinstance(n, str):
        return ord(n) if len(n) else 0
    else:
        return n

def test2(n, m):
    print(f'test2() - {n}, {m}')
    return ord(n)+m

[1] map() — Series/DataFrame 원소 단위(element-wise) 변환<hr>
- 원소별 1:1 매핑
- 원소 단위 변환

[1-1] Series.map(dict / 다른 Series / callable) : 직접 매개변수 전달 불가

In [4]:
## Series 인스턴스 생성
s = pd.Series([1, 2, 3, 4])

## (1) 내장함수 bool()로 원소 처리
#print(s.map(bool))

## (2) 람다/익명함수로 원소 처리
#print(s.map(lambda x: x**2))

# ## (3) 딕셔너리로 매핑
#print(s.map({1:'one', 2:'two', 3:'three'}))

## (4) Series에도 적용 가능 (조건부)
#print(s.map(lambda x: '짝수' if x % 2 == 0 else '홀수'))

## (5) 사용자 정의 함수
print(s.map(test))

test() - <class 'int'>, 1
test() - <class 'int'>, 2
test() - <class 'int'>, 3
test() - <class 'int'>, 4
0    1
1    2
2    3
3    4
dtype: int64


[1-2] DataFrame.map(callable, **kwargs)

In [5]:
## DF 인스턴스 생성
df = pd.DataFrame({
    'A': [1.2345, 2.983, 3.1212],
    'B': [10, 0, 30],
    'C': ['A','F','']
})

## (1) 내장함수
# print('\n[DF.map(bool)]=========')
# print(df.map(bool))

print('\n[DF.map(round, ndigits)]=========')
print(df.iloc[:, :-1].map(round, ndigits=2))
#print(df.iloc[:, :-1].map(sum))

## (2) 익명함수
# print('\n[DF.map(lambda)]=========')
# print(df.map(lambda x: x if isinstance(x, str) else x**2))

# (3) 사용자 정의 함수
# print('\n[DF.map(사용자정의함수)]=========')
# print(df.map(test))


[DF.map(round, ndigits)]=========
      A   B
0  1.23  10
1  2.98   0
2  3.12  30


[2] apply() — Series: 원소별, DataFrame: 행/열 단위 <hr>

- 반환 크기는 자유 (축소 가능)
- 행/열 단위 연산, 복잡한 함수 적용에 사용 

[2-1] Series.apply(func, args, **kwargs) : 임의의 복잡 로직이나 추가 인자, 다중 결과 처리 시

In [6]:
## Series 인스턴스 생성
s = pd.Series(['A', 'B', 'C', 'X'])

## (1) 함수로 변환
print(s.apply(lambda x: ord(x)+10))

# (2) Series에도 적용 가능 (조건부)
print(s.apply(lambda x: '대문자' if x.isupper() else '소문자'))

# (3) 사용자 정의 함수 test2(n, m) : n -- Series의 원소1개, m -- 지정
print(s.apply(test2, m=100))

0    75
1    76
2    77
3    98
dtype: int64
0    대문자
1    대문자
2    대문자
3    대문자
dtype: object
test2() - A, 100
test2() - B, 100
test2() - C, 100
test2() - X, 100
0    165
1    166
2    167
3    188
dtype: int64


[2-2] DataFrame.apply(func, axis, result_type, args, **kwargs) : 열·행 단위 집계/가공

In [7]:
## ----------------------------------------------------------
## 함수기능 : 평균, 표준편차, 최고점 값을 리스트로 반환 
## 함수이름 : row_stats
## 매개변수 : r    - 행
## 결과반환 : 행별 평균, 표준편차, 최대값
## ----------------------------------------------------------
def row_stats(r):
    x = r[["A","B"]].to_numpy()
    return [x.mean(), x.std(ddof=1), x.max()]

## ----------------------------------------------------------
## 함수기능 : 과락 계산해 행마다 '과락여부' 1개 값 반환
## 함수이름 : has_fail
## 매개변수 : r    - 행
##           cut   - 과락 점수
## 결과반환 : 행마다 '과락여부' 1개 값 반환
## ----------------------------------------------------------
def has_fail(r, cut=60):
    return (r[["A","B"]] < cut).any()

## ----------------------------------------------------------
## 함수기능 : 지정된 값의 범위로 열 스케일링 
## 함수이름 : scale_minmax
## 매개변수 : col   - 열
##           lo=0  - 최소값
##           hi=1  - 최대값
## 결과반환 : 열마다 스케일링 결과 
## ----------------------------------------------------------
def scale_minmax(col, lo=0, hi=1):
    cmin, cmax = col.min(), col.max()
    return (col - cmin) / (cmax - cmin) * (hi - lo) + lo

In [8]:
## DF 인스턴스 생성
df = pd.DataFrame({
    'A': [82, 27, 53],
    'B': [10, 70, 80]
})

## (1) DataFrame 전체 열별 합/평균
print('\n[DF.apply([np.sum, np.mean], axis=0)])=========')
print(df.apply([np.sum, np.mean], axis=0))

## (2) 각 행마다 A, B의 차 계산
print('\n[DF.apply(lambda) =========')
print(df.apply(lambda row: row['B'] - row['A'], axis=1))

## (3) 사용자정의함수 : 여러 값을 한 번에 반환 → 컬럼으로 확장
print('\n[DF.apply(row_stats,result_type="expand") =========')
resultDF = df.apply(row_stats, axis=1, result_type='expand')
resultDF.columns = ["avg","std","max"]
print(resultDF)


## (4) 사용자정의함수 : 추가 인자/키워드 인자
print('\n[DF.apply(scale_minmax) =========')
resultDF = df.apply(scale_minmax, lo=50, hi=100)
print(resultDF)


[DF.apply([np.sum, np.mean], axis=0)])=========
          A           B
sum   162.0  160.000000
mean   54.0   53.333333

[DF.apply(lambda) =========
0   -72
1    43
2    27
dtype: int64

[DF.apply(row_stats,result_type="expand") =========
    avg        std   max
0  46.0  50.911688  82.0
1  48.5  30.405592  70.0
2  66.5  19.091883  80.0

[DF.apply(scale_minmax) =========
            A           B
0  100.000000   50.000000
1   50.000000   92.857143
2   73.636364  100.000000


[3] transform() — GroupBy 또는 Series의 크기 동일 변환<hr>
- 원본 DataFrame과 동일 (행 개수 유지) 변환
- 그룹별 표준화, 평균 대비 비율 계산 등
- Series/GroupBy에 적용

In [11]:
## DF 인스턴스 생성
df2 = pd.DataFrame({
    '지점': ['서울','서울','부산','부산','대구','대구'],
    '매출': [100,150,120,180,90,210]
})

df2

,지점,매출
0,서울,100
1,서울,150
2,부산,120
3,부산,180
4,대구,90
5,대구,210


In [32]:
## [문1] 지점별 매출 평균 계산
## [해1]
print(f'[해1]---------------')
locs=df2['지점'].unique()
for loc in locs:
    ret = df2[ df2.지점 == loc]['매출'].sum()
    print(f'[{loc}] - 매출합계 : {ret}')

## [해2]
print(f'\n[해2]---------------')
df2.groupby(by='지점')['매출'].sum()



[해1]---------------
[서울] - 매출합계 : 250
[부산] - 매출합계 : 300
[대구] - 매출합계 : 300

[해2]---------------


지점
대구    300
부산    300
서울    250
Name: 매출, dtype: int64

In [ ]:
## [문1] 지점별 매출 평균 계산
## [해1]
print(f'[해1]---------------')
locs=df2['지점'].unique()
for loc in locs:
    ret = df2[ df2.지점 == loc]['매출'].sum()
    print(f'[{loc}] - 매출합계 : {ret}')

## [해2]
print(f'\n[해2]---------------')
df2.groupby(by='지점')['매출'].sum()

In [ ]:


# # 그룹별 평균 대비 비율
# df2['매출비율'] = df2.groupby('지점')['매출'].transform(lambda x: x / x.mean())
# print(df2)


# 그룹별 평균 대비 비율
df3 = df2.groupby('지점')
for key in df3.groups:
    print( key, df3.get_group(key).sum() )

# print(df2.groupby('지점').get_group('대구'))
# df2.groupby('지점').get_group('대구').agg('sum')


   지점   매출  매출비율
0  서울  100   0.8
1  서울  150   1.2
2  부산  120   0.8
3  부산  180   1.2
4  대구   90   0.6
5  대구  210   1.4
대구 지점      대구대구
매출       300
매출비율     2.0
dtype: object
부산 지점      부산부산
매출       300
매출비율     2.0
dtype: object
서울 지점      서울서울
매출       250
매출비율     2.0
dtype: object


[4] pipe() - 함수형 체이닝(functional chaining)<hr>

- DataFrame / Series 모두 사용
- 변환 결과 크기 제약 없음

In [ ]:
## 사용자 정의 함수들
def add_ratio(df):
    df['ratio'] = df['A'] / df['B']
    return df

def filter_high(df):
    return df[df['ratio'] > 0.1]

## DF 인스턴스 생성
df = pd.DataFrame({'A': [1, 3, 5], 'B': [10, 15, 20]})

# retDF = add_ratio(df)
# retDF = filter_high(retDF)


# 단계별 체인
result = ( df.pipe(add_ratio).pipe(filter_high)  )

display(result)


,A,B,ratio
1,3,15,0.20
2,5,20,0.25


In [68]:
df.columns

Index(['A', 'B', 'ratio'], dtype='object')

In [73]:
sorted(df.columns, reverse=True)

['ratio', 'B', 'A']